In [1]:
# %%writefile infer_fuse.py
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.core import Workspace, Datastore, Environment, Experiment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, StepSequence
from azureml.core.runconfig import RunConfiguration
from azureml.widgets import RunDetails

ws = Workspace.from_config()
blob_store = Datastore(ws, "workspaceblobstore")    # workspaceblobstore is the default

# Create Experiment
experiment_name = 'infer_fusion'
exp = Experiment(ws, experiment_name)

# Create Compute Target
aml_compute_target = 'vdsm-cluster-l80'  # ALL SMALL LETTER, NO underscore, 16ws long only.
aml_compute = AmlCompute(ws, aml_compute_target)

# Use an existing env which was registered to the ws
env = Environment.get(workspace=ws,name="vdsm_all",version="1")

In [2]:
# A simple pipeline structure that can run different steps in a compute cluster
from azureml.pipeline.steps import PythonScriptStep, ParallelRunStep, ParallelRunConfig
from azureml.pipeline.core.pipeline_output_dataset import PipelineOutputTabularDataset
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.data import OutputFileDatasetConfig
from azureml.pipeline.core.graph import PipelineParameter

pipeline_run_config = ParallelRunConfig(environment=env,
                                        entry_script="infer_fusion.py",
                                        error_threshold=-1,
                                        output_action="summary_only",
                                        compute_target=aml_compute,
                                        process_count_per_node=30,
                                        run_invocation_timeout=7200,
                                        node_count=2)

month_pipeline_param = PipelineParameter(name="month", default_value="2022-01")
random_n_pipeline_param = PipelineParameter(name="random_n", default_value="-1")
pv_pipeline_param = PipelineParameter(name="pv", default_value=False)
npv_pipeline_param = PipelineParameter(name="npv", default_value=True)

ds_list = []
for i in range(1,249,2):
    ds_list.append(DatasetConsumptionConfig(name="row_{}_{}".format(i, i+1), dataset=ws.datasets.get("row_{}_{}".format(i, i+1))))

ds_list.append(DatasetConsumptionConfig(name="row_249", dataset=ws.datasets.get("row_249")))

fusion_step = ParallelRunStep(
    name="fuse_for_inference",
    inputs=ds_list,
    arguments=[
        "--PV", pv_pipeline_param,
        "--NPV", npv_pipeline_param,
        "--month", month_pipeline_param,
        "--random_n", random_n_pipeline_param
    ],
    parallel_run_config=pipeline_run_config)

step_sequence = StepSequence(steps=[fusion_step])

pipeline = Pipeline(workspace=ws, steps=step_sequence)
# pipeline_run = exp.submit(pipeline)
# print('VDSM training data fusion submitted for execution')
# RunDetails(pipeline_run).show()
# pipeline_run.wait_for_completion()

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/azureml/pipeline/core/_parallel_run_step_base.py:585: UserWarning: 
ParallelRunStep requires azureml-dataset-runtime[fuse,pandas] for tabular dataset.
Please add relevant package in CondaDependencies.
  UserWarning,


In [3]:
pipeline.publish(name="infer_fusion", description="Fuse S2 data for inference", version=10)

Created step fuse_for_inference [e54f41f0][7a7d5b27-7e63-4833-99c5-2b5c4c9ff0f4], (This step will run and generate new outputs)


Pipeline(Name: infer_fusion,
Id: 978fdd72-3eb3-4ca4-bc5e-2f2413a37524,
Status: Active,
Endpoint: https://australiaeast.api.azureml.ms/pipelines/v1.0/subscriptions/7704663e-8944-4300-b951-6462d41ab638/resourceGroups/arg-rt-pacedev-spacetech/providers/Microsoft.MachineLearningServices/workspaces/aml-ws-spacetech001/PipelineRuns/PipelineSubmit/978fdd72-3eb3-4ca4-bc5e-2f2413a37524)